# 逐步构建你的深度神经网络

之前已经训练过一个2层神经网络（包含一个隐藏层）。本节将构建一个深度神经网络，拥有任意数量的隐藏层！

- 在本节中，你将实现构建深度神经网络所需的所有函数。
- 在本节中，你将使用这些函数构建一个用于图像分类的深度神经网络。

**完成本节后，你将能够：**
- 使用 ReLU 等非线性单元提升模型性能
- 构建更深的神经网络（超过1个隐藏层）
- 实现一个易于使用的神经网络类

**符号说明**：
- 上标 $[l]$ 表示第 $l$ 层相关的量。  
  例如：$a^{[L]}$ 是第 $L$ 层的激活值，$W^{[L]}$ 和 $b^{[L]}$ 是第 $L$ 层的参数。
- 上标 $(i)$ 表示第 $i$ 个样本相关的量。  
  例如：$x^{(i)}$ 是第 $i$ 个训练样本。
- 下标 $i$ 表示向量的第 $i$ 个元素。  
  例如：$a^{[l]}_i$ 表示第 $l$ 层激活值的第 $i$ 个元素。

让我们开始吧！


## 1 - 导入所需包

首先导入需要用到的所有包：

- [numpy](www.numpy.org) 是 Python 中进行科学计算的主要库，支持多维数组和矩阵运算。
- [matplotlib](http://matplotlib.org) 是用于绘制各种图表的库，方便数据可视化。
- dnn_utils 提供了需要用到的一些辅助函数。
- testCases 提供了一些测试用例，用于检测你所写函数的正确性。
- np.random.seed(1) 用于固定随机数种子，保证所有随机函数调用的结果一致。


In [18]:
import numpy as np                              # 导入 numpy，用于科学计算和数组操作
import h5py                                    # 导入 h5py，用于读取和写入 HDF5 格式的数据文件
import matplotlib.pyplot as plt                # 导入 matplotlib.pyplot，用于绘图和数据可视化
from testCases_v2 import *                      # 导入 testCases_v2 中的所有测试用例函数，方便后续测试代码正确性
from dnn_utils_v2 import sigmoid, sigmoid_backward, relu, relu_backward  
# 从 dnn_utils_v2 导入激活函数及其反向传播函数，包括 sigmoid 和 relu 及其导数

# 在 Jupyter Notebook 中直接显示 matplotlib 绘制的图形
%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0)    # 设置默认绘图尺寸为 5x4 英寸
plt.rcParams['image.interpolation'] = 'nearest' # 设置图像插值方式为最近邻插值，保证图像清晰
plt.rcParams['image.cmap'] = 'gray'            # 设置图像的默认色彩映射为灰度图

# 加载 IPython 扩展 autoreload，自动重新加载修改过的模块
# 设置所有导入的模块自动重新加载，便于调试时代码修改生效
%load_ext autoreload
%autoreload 2

np.random.seed(1)                              # 固定随机数种子为1，保证所有随机数操作结果一致，方便复现


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 2 - 大纲

为了构建你的神经网络，你将实现多个“辅助函数”。这些辅助函数将在下节中用于构建一个两层神经网络和一个多层（L层）神经网络。每个小的辅助函数都将有详细的说明，指导你完成必要的步骤。以下是本节的整体流程，你将：

- 初始化两层网络和 L 层神经网络的参数。
- 实现前向传播模块（见下图中紫色部分）。
  - 完成每层前向传播中的 LINEAR 部分（计算 $Z^{[l]}$）。
  - ACTIVATION 函数（relu/sigmoid）。
  - 将上述两步结合，形成一个新的 [LINEAR->ACTIVATION] 前向函数。
  - 将 [LINEAR->RELU] 前向函数堆叠 L-1 次（对应第1层到第L-1层），并在最后添加一个 [LINEAR->SIGMOID] 前向函数（对应最后一层 $L$），从而实现新的 `L_model_forward` 函数。
- 计算损失（loss）。
- 实现反向传播模块（见下图中红色部分）。
  - 完成每层反向传播中的 LINEAR 部分。
  - ACTIVATION 函数的梯度计算函数（relu_backward/sigmoid_backward）。
  - 将上述两步结合，形成新的 [LINEAR->ACTIVATION] 反向函数。
  - 将 [LINEAR->RELU] 反向函数堆叠 L-1 次，并在最后添加 [LINEAR->SIGMOID] 反向函数，从而实现新的 `L_model_backward` 函数。
- 最后，更新参数。

<img src="images/final outline.png" style="width:800px;height:500px;">
<caption><center> **图 1**</center></caption><br>

**注意**：每个前向函数都有对应的反向函数。因此在前向传播的每一步，你都需要将一些值存入缓存（cache），以便后续计算梯度时使用。在反向传播模块中，将利用缓存计算梯度。本节将逐步指导你完成这些步骤。


## 3 - 参数初始化

你将编写两个辅助函数来初始化模型参数。第一个函数用于初始化两层神经网络的参数，第二个函数将该过程推广到 $L$ 层网络。

### 3.1 - 两层神经网络

**练习**：创建并初始化两层神经网络的参数。

**说明**：
- 网络结构为：*LINEAR -> RELU -> LINEAR -> SIGMOID*。
- 权重矩阵使用随机初始化，方法为 `np.random.randn(形状) * 0.01`，确保形状正确。
- 偏置向量使用全零初始化，方法为 `np.zeros(形状)`。


In [19]:
# 初始化神经网络参数的函数

def initialize_parameters(n_x, n_h, n_y):
    """
    参数说明：
    n_x -- 输入层神经元数量（特征数）
    n_h -- 隐藏层神经元数量
    n_y -- 输出层神经元数量（分类数）
    
    返回值：
    parameters -- 字典，包含以下参数：
                  W1 -- 第一层权重矩阵，形状为 (n_h, n_x)
                  b1 -- 第一层偏置向量，形状为 (n_h, 1)
                  W2 -- 第二层权重矩阵，形状为 (n_y, n_h)
                  b2 -- 第二层偏置向量，形状为 (n_y, 1)
    """
    
    np.random.seed(1)  # 固定随机数种子，确保初始化结果可复现
    
    # 用均值为0，标准差为0.01的高斯分布随机初始化第一层权重矩阵，形状为 (隐藏层节点数, 输入层节点数)
    W1 = np.random.randn(n_h, n_x) * 0.01
    
    # 第一层偏置初始化为全零向量，形状为 (隐藏层节点数, 1)
    b1 = np.zeros((n_h, 1))
    
    # 第二层权重矩阵随机初始化，形状为 (输出层节点数, 隐藏层节点数)
    W2 = np.random.randn(n_y, n_h) * 0.01
    
    # 第二层偏置初始化为全零向量，形状为 (输出层节点数, 1)
    b2 = np.zeros((n_y, 1))
    
    # 使用断言确保参数形状正确，防止后续运算错误
    assert(W1.shape == (n_h, n_x))  # W1应为隐藏层节点数行，输入层节点数列
    assert(b1.shape == (n_h, 1))    # b1应为隐藏层节点数行，1列
    assert(W2.shape == (n_y, n_h))  # W2应为输出层节点数行，隐藏层节点数列
    assert(b2.shape == (n_y, 1))    # b2应为输出层节点数行，1列
    
    # 将所有参数封装到字典中，方便后续调用和管理
    parameters = {
        "W1": W1,
        "b1": b1,
        "W2": W2,
        "b2": b2
    }
    
    return parameters


In [45]:
# 使用初始化参数函数初始化一个简单的2层神经网络，输入层2个节点，隐藏层2个节点，输出层1个节点
parameters = initialize_parameters(2, 2, 1)

# 打印第一层权重矩阵W1
print("W1 = " + str(parameters["W1"]))

# 打印第一层偏置向量b1
print("b1 = " + str(parameters["b1"]))

# 打印第二层权重矩阵W2
print("W2 = " + str(parameters["W2"]))

# 打印第二层偏置向量b2
print("b2 = " + str(parameters["b2"]))


W1 = [[ 0.01624345 -0.00611756]
 [-0.00528172 -0.01072969]]
b1 = [[0.]
 [0.]]
W2 = [[ 0.00865408 -0.02301539]]
b2 = [[0.]]


### 3.2 - 多层（L层）神经网络

深层 L 层神经网络的初始化更复杂，因为有更多的权重矩阵和偏置向量。在完成 `initialize_parameters_deep` 函数时，要确保各层参数的维度匹配。回顾一下，$n^{[l]}$ 表示第 $l$ 层的神经元个数。举例来说，若输入数据 $X$ 的大小为 $(12288, 209)$（样本数 $m=209$），那么：

<table style="width:100%">

<tr>
    <td> </td>
    <td>**权重矩阵 W 的形状**</td>
    <td>**偏置向量 b 的形状**</td>
    <td>**激活计算**</td>
    <td>**激活值的形状**</td>
</tr>

<tr>
    <td>**第1层**</td>
    <td>$(n^{[1]}, 12288)$</td>
    <td>$(n^{[1]}, 1)$</td>
    <td>$Z^{[1]} = W^{[1]} X + b^{[1]}$</td>
    <td>$(n^{[1]}, 209)$</td>
</tr>

<tr>
    <td>**第2层**</td>
    <td>$(n^{[2]}, n^{[1]})$</td>
    <td>$(n^{[2]}, 1)$</td>
    <td>$Z^{[2]} = W^{[2]} A^{[1]} + b^{[2]}$</td>
    <td>$(n^{[2]}, 209)$</td>
</tr>

<tr>
    <td> $\vdots$ </td>
    <td> $\vdots$ </td>
    <td> $\vdots$ </td>
    <td> $\vdots$ </td>
    <td> $\vdots$ </td>
</tr>

<tr>
    <td>**第L-1层**</td>
    <td>$(n^{[L-1]}, n^{[L-2]})$</td>
    <td>$(n^{[L-1]}, 1)$</td>
    <td>$Z^{[L-1]} = W^{[L-1]} A^{[L-2]} + b^{[L-1]}$</td>
    <td>$(n^{[L-1]}, 209)$</td>
</tr>

<tr>
    <td>**第L层**</td>
    <td>$(n^{[L]}, n^{[L-1]})$</td>
    <td>$(n^{[L]}, 1)$</td>
    <td>$Z^{[L]} = W^{[L]} A^{[L-1]} + b^{[L]}$</td>
    <td>$(n^{[L]}, 209)$</td>
</tr>

</table>

注意，在 Python 中计算 $W X + b$ 时会自动广播。例如，若：

$$ W = \begin{bmatrix}
    j & k & l \\
    m & n & o \\
    p & q & r 
\end{bmatrix}, \quad
X = \begin{bmatrix}
    a & b & c \\
    d & e & f \\
    g & h & i 
\end{bmatrix}, \quad
b = \begin{bmatrix}
    s \\
    t \\
    u
\end{bmatrix} \tag{2}$$

则：

$$
WX + b = \begin{bmatrix}
    (ja + kd + lg) + s & (jb + ke + lh) + s & (jc + kf + li) + s \\
    (ma + nd + og) + t & (mb + ne + oh) + t & (mc + nf + oi) + t \\
    (pa + qd + rg) + u & (pb + qe + rh) + u & (pc + qf + ri) + u
\end{bmatrix} \tag{3}
$$


**练习**：实现多层（L层）神经网络的参数初始化。

**说明**：
- 网络结构为 *[LINEAR -> RELU] $ \times$ (L-1) -> LINEAR -> SIGMOID*，即前 $L-1$ 层使用 ReLU 激活函数，最后一层使用 sigmoid 激活函数。
- 权重矩阵使用随机初始化，方法为 `np.random.randn(形状) * 0.01`。
- 偏置向量使用零初始化，方法为 `np.zeros(形状)`。
- 将每层神经元个数存储在列表 `layer_dims` 中。例如，“平面数据分类模型”的案例 `layer_dims` 为 `[2, 4, 1]`，表示输入层2个单元，1个隐藏层4个单元，输出层1个单元。对应权重和偏置的形状为：`W1` 是 (4, 2)，`b1` 是 (4, 1)，`W2` 是 (1, 4)，`b2` 是 (1, 1)。  
  现在你需要将该过程推广到 $L$ 层神经网络。

- 以下是单层神经网络（$L=1$）的示例代码，可供参考：
```python
if L == 1:
    parameters["W" + str(L)] = np.random.randn(layer_dims[1], layer_dims[0]) * 0.01
    parameters["b" + str(L)] = np.zeros((layer_dims[1], 1))


In [21]:
def initialize_parameters_deep(layer_dims):
    """
    功能：
    初始化一个深层神经网络的所有参数（权重矩阵和偏置向量）。
    
    参数：
    layer_dims -- Python 列表，包含网络中每一层的神经元数量
                  例如 [5, 4, 3] 表示：
                  输入层：5个神经元
                  隐藏层：4个神经元
                  输出层：3个神经元
    
    返回：
    parameters -- Python 字典，包含每一层的参数：
                  "W1", "b1", ..., "WL", "bL"
                  Wl -- 权重矩阵，形状为 (layer_dims[l], layer_dims[l-1])
                  bl -- 偏置向量，形状为 (layer_dims[l], 1)
    """
    
    np.random.seed(3)  # 固定随机种子，保证实验结果可复现
    parameters = {}    # 用字典存储所有参数
    L = len(layer_dims)  # 网络的层数（包含输入层）
    
    # 从第一层权重（连接输入层与第一隐藏层）开始依次初始化
    for l in range(1, L):
        # 权重矩阵 Wl 初始化为服从均值0、标准差0.01的高斯分布随机值
        # 形状：(当前层神经元数量, 前一层神经元数量)
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
          
        # 偏置向量 bl 初始化为全零
        # 形状：(当前层神经元数量, 1)
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        
        # 使用断言检查形状是否正确，防止后续矩阵运算出错
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

    return parameters


In [44]:
# 使用多层初始化函数初始化一个3层神经网络，层节点分别为5, 4, 3
parameters = initialize_parameters_deep([5, 4, 3])

# 打印第一层权重矩阵W1
print("W1 = " + str(parameters["W1"]))

# 打印第一层偏置向量b1
print("b1 = " + str(parameters["b1"]))

# 打印第二层权重矩阵W2
print("W2 = " + str(parameters["W2"]))

# 打印第二层偏置向量b2
print("b2 = " + str(parameters["b2"]))


W1 = [[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]]
W2 = [[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]
b2 = [[0.]
 [0.]
 [0.]]


## 4 - 前向传播模块

### 4.1 - 线性前向（Linear Forward）  
参数初始化完成后，接下来你将实现前向传播模块。首先完成几个基础函数，后续构建完整模型时将用到。你需要按顺序完成以下三个函数：

- LINEAR  
- LINEAR -> ACTIVATION，其中 ACTIVATION 为 ReLU 或 Sigmoid。  
- [LINEAR -> RELU] $\times$ (L-1) -> LINEAR -> SIGMOID（完整模型）

线性前向模块（对所有样本向量化计算）实现如下公式：

$$Z^{[l]} = W^{[l]}A^{[l-1]} + b^{[l]} \tag{4}$$

其中，$A^{[0]} = X$（输入数据）。

**练习**：实现前向传播中的线性部分。

**提示**：  
该模块数学表达为 $Z^{[l]} = W^{[l]} A^{[l-1]} + b^{[l]}$。你可以用 `np.dot()` 来做矩阵乘法。如果维度不匹配，建议打印 `W.shape` 进行调试。


In [23]:
def linear_forward(A, W, b):
    """
    功能：
    实现神经网络中某一层的**线性部分前向传播**（不包含激活函数）。
    即：Z = W*A + b
    
    参数：
    A -- 前一层的激活值（或者是输入数据 X）
         形状为 (上一层神经元数量, 样本数量 m)
    W -- 当前层的权重矩阵
         形状为 (当前层神经元数量, 上一层神经元数量)
    b -- 当前层的偏置向量
         形状为 (当前层神经元数量, 1)
    
    返回：
    Z -- 当前层的线性计算输出（激活函数的输入，也叫 pre-activation 参数）
    cache -- 元组 (A, W, b)，存储这些值以便反向传播使用
    """
    
    # 矩阵相乘 + 偏置广播
    Z = np.dot(W, A) + b  # Z 的形状是 (当前层神经元数量, 样本数量 m)
    
    # 确保计算结果 Z 的维度正确
    assert(Z.shape == (W.shape[0], A.shape[1]))
    
    # 存储中间值以便反向传播
    cache = (A, W, b)
    
    return Z, cache


In [43]:
# 获取测试输入：激活值A，权重W，偏置b，用于测试线性前向传播函数
A, W, b = linear_forward_test_case()

# 调用线性前向传播函数，计算线性输出Z，并获取缓存linear_cache
Z, linear_cache = linear_forward(A, W, b)

# 打印线性层输出Z
print("Z = " + str(Z))


Z = [[ 3.26295337 -1.23429987]]


## 上述代码中用到了元组，在此，强调一下几种数据类型的区别

## Python 列表、字典、元组、集合 对比表

| 数据类型 | 存储方式 | 是否有序 | 是否可变 | 是否允许重复元素 | 访问方式 | 常用场景 | 示例 |
|----------|----------|----------|----------|------------------|----------|----------|------|
| **列表（list）** | 有序元素集合 | ✅ 有序 | ✅ 可变 | 允许 | 通过索引访问，如 `my_list[0]` | 存储、遍历、批量修改数据 | `[1, 2, 3]` |
| **字典（dict）** | 键值对（key-value） | Python 3.7+ 有序（之前无序） | ✅ 可变 | 键不允许重复，值可重复 | 通过键访问，如 `my_dict["name"]` | 映射关系、快速查找 | `{"name": "Tom", "age": 25}` |
| **元组（tuple）** | 有序元素集合 | ✅ 有序 | ❌ 不可变 | 允许 | 通过索引访问，如 `my_tuple[0]` | 保护数据、多返回值、作字典键 | `(1, 2, 3)` |
| **集合（set）** | 无序唯一元素集合 | ❌ 无序 | ✅ 可变（元素必须不可变） | 不允许 | 不能通过索引，只能迭代或判断元素存在 | 去重、集合运算 | `{1, 2, 3}` |


### 4.2 - 线性-激活前向（Linear-Activation Forward）

你将使用两种激活函数：

- **Sigmoid**：$\sigma(Z) = \sigma(W A + b) = \frac{1}{1 + e^{-(W A + b)}}$。  
  我们提供了 `sigmoid` 函数。该函数返回两个值：激活值 "`A`" 和一个包含 "`Z`" 的 "`cache`"（该缓存将用于对应的反向函数）。调用示例如下：
```python
A, activation_cache = sigmoid(Z)

- **ReLU**：ReLU 的数学公式是 $A = RELU(Z) = \max(0, Z)$。我们提供了 `relu` 函数。该函数返回两个值：激活值 "`A`" 和包含 "`Z`" 的 "`cache`"（这个缓存会被用于对应的反向传播函数）。使用示例如下：
```python
A, activation_cache = relu(Z)


为了方便起见，你将把两个函数（线性层和激活层）合并成一个函数（LINEAR->ACTIVATION）。  
因此，你需要实现一个函数，先执行线性前向，再执行激活前向。

**练习**：实现 *LINEAR->ACTIVATION* 层的前向传播。数学表达为：  
$$A^{[l]} = g(Z^{[l]}) = g(W^{[l]}A^{[l-1]} + b^{[l]})$$  
其中激活函数 $g$ 可以是 sigmoid() 或 relu()。请调用你之前实现的 `linear_forward()` 函数，并结合正确的激活函数完成本任务。


In [25]:
def linear_activation_forward(A_prev, W, b, activation):
    """
    功能：
    实现神经网络中一层的前向传播，包括线性部分和激活函数部分。
    即：先计算 Z = W*A_prev + b，再对 Z 应用激活函数（sigmoid 或 relu）
    
    参数：
    A_prev -- 来自前一层的激活值，形状为（上一层神经元数量, 样本数量）
    W -- 当前层权重矩阵，形状为（当前层神经元数量, 上一层神经元数量）
    b -- 当前层偏置向量，形状为（当前层神经元数量, 1）
    activation -- 激活函数类型，字符串，取值为 "sigmoid" 或 "relu"
    
    返回：
    A -- 当前层激活后的输出（激活值），形状为（当前层神经元数量, 样本数量）
    cache -- 用于反向传播的缓存，包含线性缓存和激活缓存
    """
    
    if activation == "sigmoid":
        # 调用线性前向函数计算 Z，并缓存线性部分的中间变量
        Z, linear_cache = linear_forward(A_prev, W, b)
        # 对 Z 使用 sigmoid 激活函数，并缓存激活过程中的变量
        A, activation_cache = sigmoid(Z)
    
    elif activation == "relu":
        # 调用线性前向函数计算 Z，并缓存线性部分的中间变量
        Z, linear_cache = linear_forward(A_prev, W, b)
        # 对 Z 使用 relu 激活函数，并缓存激活过程中的变量
        A, activation_cache = relu(Z)
    
    # 确认输出 A 的形状正确，应该是 (当前层神经元数量, 样本数)
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    
    # 将线性缓存和激活缓存打包返回，方便后续反向传播使用
    cache = (linear_cache, activation_cache)

    return A, cache


In [42]:
# 获取测试输入：上一层激活值A_prev，权重W，偏置b，用于测试带激活函数的线性前向传播函数
A_prev, W, b = linear_activation_forward_test_case()

# 使用sigmoid激活函数进行前向传播，并打印输出激活值A
A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation="sigmoid")
print("With sigmoid: A = " + str(A))

# 使用ReLU激活函数进行前向传播，并打印输出激活值A
A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation="relu")
print("With ReLU: A = " + str(A))


With sigmoid: A = [[0.96890023 0.11013289]]
With ReLU: A = [[3.43896131 0.        ]]


**注意**：在深度学习中，“[LINEAR->ACTIVATION]” 被视为神经网络中的一个整体层，而不是两个独立的层。


### d) L层模型（L-Layer Model）

为了更方便地实现 $L$ 层神经网络，你需要实现一个函数，该函数会调用之前写的 `linear_activation_forward`（使用 ReLU 激活）$L-1$ 次，然后再调用一次 `linear_activation_forward`（使用 Sigmoid 激活）。

<img src="images/model_architecture_kiank.png" style="width:600px;height:300px;">
<caption><center> **图 2** ：*[LINEAR -> RELU] $\times$ (L-1) -> LINEAR -> SIGMOID* 模型结构示意</center></caption><br>

**练习**：实现上述模型的前向传播。

**说明**：代码中变量 `AL` 表示 $A^{[L]} = \sigma(Z^{[L]}) = \sigma(W^{[L]} A^{[L-1]} + b^{[L]})$，也就是预测值 $\hat{Y}$。

**提示**：
- 使用你之前写好的函数完成
- 用 for 循环重复执行 [LINEAR->RELU] $L-1$ 次
- 不要忘了将每层计算过程中的缓存（cache）存入列表 `caches` 中。列表添加元素用 `list.append(c)`。


In [29]:
def L_model_forward(X, parameters):
    """
    功能：
    实现整个深层神经网络的前向传播过程，结构为：
    [LINEAR -> RELU] × (L-1) 层 + LINEAR -> SIGMOID 最后一层
    
    参数：
    X -- 输入数据，形状为（输入层大小，样本数）
    parameters -- initialize_parameters_deep() 初始化得到的参数字典，包含所有层的 W 和 b
    
    返回：
    AL -- 最后一层激活值（预测输出），形状为 (1, 样本数)
    caches -- 存储每一层线性和激活缓存的列表，供反向传播使用
    """

    caches = []           # 用于存储所有层的缓存
    A = X                 # 初始化激活值为输入数据
    L = len(parameters) // 2  # 网络层数（参数字典中权重和偏置数量各一半）

    # 前向传播：第1层到第L-1层，激活函数均为 ReLU
    for l in range(1, L):
        A_prev = A
        # 计算当前层的激活值和缓存，激活函数选relu
        A, cache = linear_activation_forward(
            A_prev, 
            parameters['W' + str(l)], 
            parameters['b' + str(l)], 
            activation="relu"
        )
        caches.append(cache)  # 保存当前层缓存
    
    # 最后一层使用 sigmoid 激活函数，输出层激活值 AL
    AL, cache = linear_activation_forward(
        A, 
        parameters['W' + str(L)], 
        parameters['b' + str(L)], 
        activation="sigmoid"
    )
    caches.append(cache)  # 保存最后一层缓存
    
    # 确保输出 AL 形状为 (1, 样本数)
    assert(AL.shape == (1, X.shape[1]))
            
    return AL, caches


In [41]:
# 获取测试输入X和参数parameters，用于测试多层网络的前向传播函数
X, parameters = L_model_forward_test_case()

# 执行多层前向传播，得到最终激活值AL和缓存caches
AL, caches = L_model_forward(X, parameters)

# 打印输出层激活值AL
print("AL = " + str(AL))

# 打印缓存列表的长度，应该等于网络层数L
print("Length of caches list = " + str(len(caches)))


AL = [[0.17007265 0.2524272 ]]
Length of caches list = 2


很好！现在你已经实现了完整的前向传播函数，它以输入 $X$ 为起点，输出一个行向量 $A^{[L]}$，其中包含你的预测结果。同时，它将所有中间计算值保存在变量 `caches` 中。利用 $A^{[L]}$，你就可以计算预测的代价函数了。


## 5 - 代价函数（Cost function）

现在你将实现前向和反向传播中的代价计算。计算代价是为了检查模型是否在有效学习。

**练习**：根据以下公式计算交叉熵代价函数 $J$：

$$
J = -\frac{1}{m} \sum_{i=1}^{m} \left( y^{(i)} \log\left(a^{[L](i)}\right) + (1 - y^{(i)}) \log\left(1 - a^{[L](i)}\right) \right) \tag{7}
$$


In [33]:
def compute_cost(AL, Y):
    """
    功能：
    计算使用交叉熵损失函数的成本（cost），公式参考课件中的方程（7）。
    
    参数：
    AL -- 预测标签的概率向量，形状为 (1, 样本数量)
    Y -- 真实标签向量（0或1），形状为 (1, 样本数量)
    
    返回：
    cost -- 交叉熵损失的标量值
    """
    
    m = Y.shape[1]  # 样本数量
    
    # 按照交叉熵损失公式计算总损失
    # np.sum 沿样本维度求和，axis=1 保持二维，keepdims=True 保持列向量形状
    cost = -1 / m * np.sum(Y * np.log(AL) + (1 - Y) * np.log(1 - AL), axis=1, keepdims=True)
    
    # 将 cost 从 [[标量]] 变成 标量
    cost = np.squeeze(cost)
    
    # 确保 cost 是一个标量（无维度）
    assert(cost.shape == ())
    
    return cost


In [40]:
# 设置测试输入数据，用于验证 compute_cost 函数的正确性
Y, AL = compute_cost_test_case()

# 调用计算成本函数，并打印结果
print("cost = " + str(compute_cost(AL, Y)))


cost = 0.41493159961539694


## 6 - 反向传播模块

和前向传播一样，你将实现一些辅助函数来完成反向传播。记住，反向传播用于计算损失函数对参数的梯度。

**提醒**：  
<img src="images/backprop_kiank.png" style="width:650px;height:250px;">  
<caption><center> **图 3** ：*LINEAR->RELU->LINEAR->SIGMOID* 的前向和反向传播 <br> *紫色方块表示前向传播，红色方块表示反向传播。*</center></caption>

<!-- 
对于熟悉微积分的同学（完成本作业不强制需要），可以用链式法则推导二层网络中损失 $\mathcal{L}$ 对 $z^{[1]}$ 的导数：

$$
\frac{d \mathcal{L}(a^{[2]}, y)}{dz^{[1]}} = \frac{d\mathcal{L}(a^{[2]}, y)}{da^{[2]}} \cdot \frac{da^{[2]}}{dz^{[2]}} \cdot \frac{dz^{[2]}}{da^{[1]}} \cdot \frac{da^{[1]}}{dz^{[1]}} \tag{8}
$$

计算梯度 $dW^{[1]} = \frac{\partial L}{\partial W^{[1]}}$ 时，使用链式法则：
$$
dW^{[1]} = dz^{[1]} \times \frac{\partial z^{[1]}}{\partial W^{[1]}}
$$

同理，计算 $db^{[1]} = \frac{\partial L}{\partial b^{[1]}}$：
$$
db^{[1]} = dz^{[1]} \times \frac{\partial z^{[1]}}{\partial b^{[1]}}
$$

这就是我们说的**反向传播**。
!-->

现在，和前向传播对应，反向传播的实现分为三步：  
- LINEAR 层的反向传播  
- LINEAR -> ACTIVATION 层的反向传播，其中 ACTIVATION 是 ReLU 或 Sigmoid 的导数  
- 将 [LINEAR -> RELU] 重复 L-1 次，最后加一个 [LINEAR -> SIGMOID] 的反向传播（整个模型）


### 6.1 - 线性部分的反向传播（Linear backward）

对于第 $l$ 层，其线性部分为：  
$$Z^{[l]} = W^{[l]} A^{[l-1]} + b^{[l]}$$  
（后面跟随激活函数）

假设你已经计算出了导数  
$$dZ^{[l]} = \frac{\partial \mathcal{L}}{\partial Z^{[l]}}$$  
现在你需要计算  
$$(dW^{[l]}, db^{[l]}, dA^{[l-1]})$$。

<img src="images/linearback_kiank.png" style="width:250px;height:300px;">
<caption><center> **图 4** </center></caption>

这三个输出$(dW^{[l]}, db^{[l]}, dA^{[l-1]})$ 是基于输入 $dZ^{[l]}$ 计算得到的。具体公式如下：

$$ dW^{[l]} = \frac{\partial \mathcal{L}}{\partial W^{[l]}} = \frac{1}{m} dZ^{[l]} A^{[l-1] T} \tag{8}$$

$$ db^{[l]} = \frac{\partial \mathcal{L}}{\partial b^{[l]}} = \frac{1}{m} \sum_{i=1}^{m} dZ^{[l](i)} \tag{9}$$

$$ dA^{[l-1]} = \frac{\partial \mathcal{L}}{\partial A^{[l-1]}} = W^{[l] T} dZ^{[l]} \tag{10}$$


**练习**：使用以上三个公式实现 `linear_backward()` 函数。


In [37]:
def linear_backward(dZ, cache):
    """
    功能：
    实现单层神经网络的线性部分反向传播。
    计算损失关于上一层激活值、当前层权重和偏置的梯度。
    
    参数：
    dZ -- 当前层线性输出 Z 的梯度，形状与 Z 相同，(当前层神经元数量, 样本数)
    cache -- 包含前向传播时的变量元组 (A_prev, W, b)
             A_prev: 前一层激活值，形状为 (上一层神经元数量, 样本数)
             W: 当前层权重矩阵，形状为 (当前层神经元数量, 上一层神经元数量)
             b: 当前层偏置向量，形状为 (当前层神经元数量, 1)
    
    返回：
    dA_prev -- 损失对上一层激活值 A_prev 的梯度，形状与 A_prev 相同
    dW -- 损失对当前层权重 W 的梯度，形状与 W 相同
    db -- 损失对当前层偏置 b 的梯度，形状与 b 相同
    """
    
    # 从缓存中取出前一层激活值、当前层权重和偏置
    A_prev, W, b = cache
    
    m = A_prev.shape[1]  # 样本数量
    
    # 计算权重梯度 dW，形状 (当前层神经元数, 上一层神经元数)
    # 公式：dW = (1/m) * dZ * A_prev^T
    dW = 1 / m * np.dot(dZ, A_prev.T)
    
    # 计算偏置梯度 db，形状 (当前层神经元数, 1)
    # 公式：db = (1/m) * sum(dZ) 对所有样本求和，保持维度
    db = 1 / m * np.sum(dZ, axis=1, keepdims=True)
    
    # 计算上一层激活值的梯度 dA_prev，形状 (上一层神经元数, 样本数)
    # 公式：dA_prev = W^T * dZ
    dA_prev = np.dot(W.T, dZ)
    
    # 断言确保维度正确
    assert(dA_prev.shape == A_prev.shape)
    assert(dW.shape == W.shape)
    assert(db.shape == b.shape)
    
    return dA_prev, dW, db


In [39]:
# 设置测试输入数据，用于验证 linear_backward 函数的正确性
dZ, linear_cache = linear_backward_test_case()

# 调用线性反向传播函数，计算梯度
dA_prev, dW, db = linear_backward(dZ, linear_cache)

# 输出计算得到的梯度结果
print("dA_prev = " + str(dA_prev))  # 上一层激活值梯度
print("dW = " + str(dW))            # 权重梯度
print("db = " + str(db))            # 偏置梯度


dA_prev = [[ 0.51822968 -0.19517421]
 [-0.40506361  0.15255393]
 [ 2.37496825 -0.89445391]]
dW = [[-0.10076895  1.40685096  1.64992505]]
db = [[0.50629448]]


### 6.2 - 线性层激活函数反向传播（Linear-Activation backward）

接下来，你将创建一个函数，合并两个辅助函数：**`linear_backward`** 和激活函数的反向传播，命名为 **`linear_activation_backward`**。

为了帮助你实现 `linear_activation_backward`，我们提供了两个激活函数的反向传播函数：
- **`sigmoid_backward`**：实现 SIGMOID 单元的反向传播。使用方法：
```python
dZ = sigmoid_backward(dA, activation_cache)


- **`relu_backward`**: 实现 ReLU 单元的反向传播。你可以如下调用:

```python
dZ = relu_backward(dA, activation_cache)
```

如果 $g(.)$ 是激活函数, 
那么`sigmoid_backward` 和 `relu_backward` 计算的是 $$dZ^{[l]} = dA^{[l]} * g'(Z^{[l]}) \tag{11}$$.  

**练习**: 实现 LINEAR->ACTIVATION 层的反向传播.

In [48]:
def linear_activation_backward(dA, cache, activation):
    """
    功能：
    实现带激活函数的线性层的反向传播。
    根据传入的激活函数类型，先计算激活函数的导数dZ，再计算线性部分的梯度dW, db, dA_prev。
    
    参数：
    dA -- 当前层激活值的梯度，形状与当前层激活值相同
    cache -- 元组，包含(linear_cache, activation_cache)
             linear_cache：线性部分缓存（A_prev, W, b）
             activation_cache：激活函数计算时缓存的Z值
    activation -- 激活函数类型，字符串，取值为 "sigmoid" 或 "relu"
    
    返回：
    dA_prev -- 损失对上一层激活值的梯度，形状与上一层激活值相同
    dW -- 损失对当前层权重W的梯度，形状与W相同
    db -- 损失对当前层偏置b的梯度，形状与b相同
    """
    
    # 拆包缓存
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        # 计算ReLU激活函数的梯度 dZ
        dZ = relu_backward(dA, activation_cache)
        # 计算线性部分的反向传播梯度 dA_prev, dW, db
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
        
    elif activation == "sigmoid":
        # 计算Sigmoid激活函数的梯度 dZ
        dZ = sigmoid_backward(dA, activation_cache)
        # 计算线性部分的反向传播梯度 dA_prev, dW, db
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    
    return dA_prev, dW, db


In [49]:
# 获取测试数据，测试带激活函数的线性反向传播函数
AL, linear_activation_cache = linear_activation_backward_test_case()

# 使用sigmoid激活函数计算反向传播的梯度
dA_prev, dW, db = linear_activation_backward(AL, linear_activation_cache, activation="sigmoid")
print("sigmoid:")
print("dA_prev = " + str(dA_prev))  # 上一层激活值的梯度
print("dW = " + str(dW))            # 权重梯度
print("db = " + str(db) + "\n")    # 偏置梯度

# 使用ReLU激活函数计算反向传播的梯度
dA_prev, dW, db = linear_activation_backward(AL, linear_activation_cache, activation="relu")
print("relu:")
print("dA_prev = " + str(dA_prev))  # 上一层激活值的梯度
print("dW = " + str(dW))            # 权重梯度
print("db = " + str(db))            # 偏置梯度


sigmoid:
dA_prev = [[ 0.11017994  0.01105339]
 [ 0.09466817  0.00949723]
 [-0.05743092 -0.00576154]]
dW = [[ 0.10266786  0.09778551 -0.01968084]]
db = [[-0.05729622]]

relu:
dA_prev = [[ 0.44090989 -0.        ]
 [ 0.37883606 -0.        ]
 [-0.2298228   0.        ]]
dW = [[ 0.44513824  0.37371418 -0.10478989]]
db = [[-0.20837892]]


### 6.3 - L-模型反向传播（L-Model Backward）

现在你将实现整个神经网络的反向传播函数。回顾你之前实现的 `L_model_forward` 函数，在每次迭代中，你都存储了一个缓存（cache），其中包含了 (X, W, b 和 z)。在反向传播模块中，你将使用这些变量来计算梯度。

因此，在 `L_model_backward` 函数中，你将从第 $L$ 层开始，向后迭代整个隐藏层。在每一步中，你都将使用第 $l$ 层缓存的值来执行该层的反向传播。下图展示了反向传播的过程。

<img src="images/mn_backward.png" style="width:450px;height:300px;">
<caption><center>  **图 5**：反向传播过程  </center></caption>

---

**反向传播初始化**：  
为了对该网络执行反向传播，我们知道输出为：  
$A^{[L]} = \sigma(Z^{[L]})$。  
你的代码需要计算 `dAL`，也就是：  
$$dAL = \frac{\partial \mathcal{L}}{\partial A^{[L]}}$$  
使用以下公式（这是利用微积分推导而来，不需要深入理解其数学细节）：

```python
dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL)) # 成本函数对 AL 的导数


你可以接着使用这个激活后的梯度 `dAL` 向后传播。如图 5 所示，你现在可以将 `dAL` 输入到你已经实现的 `LINEAR->SIGMOID` 反向传播函数中（该函数会使用由 `L_model_forward` 函数存储的缓存值）。

接下来，你需要使用一个 `for` 循环，遍历其余所有层，使用 `LINEAR->RELU` 的反向传播函数来逐层传播。

你应当将每一层的 `dA`、`dW` 和 `db` 保存到一个名为 `grads` 的字典中。可以使用下面的格式进行存储：

$$grads["dW" + str(l)] = dW^{[l]} \tag{15}$$

例如，当 $l=3$ 时，会将 $dW^{[3]}$ 存储为 `grads["dW3"]`。

---

**练习**：实现 *[LINEAR->RELU] $\times$ (L-1) -> LINEAR -> SIGMOID* 模型的反向传播过程。


In [59]:
def L_model_backward(AL, Y, caches):
    """
    功能：
    实现多层神经网络的反向传播，网络结构为：
    [LINEAR->RELU] * (L-1) -> LINEAR -> SIGMOID
    
    参数：
    AL -- 前向传播输出的预测概率，形状为(1, 样本数)
    Y -- 真实标签向量，形状与AL相同
    caches -- 包含每层缓存的列表，顺序对应前向传播：
              前L-1层的线性-RELU缓存，最后一层的线性-SIGMOID缓存
    
    返回：
    grads -- 字典，包含每层的梯度信息：
             grads["dA"+str(l)] -- 第l层激活值的梯度
             grads["dW"+str(l)] -- 第l层权重的梯度
             grads["db"+str(l)] -- 第l层偏置的梯度
    """

    grads = {}                   # 用于存储所有层的梯度
    L = len(caches)              # 网络层数（含输出层）
    m = AL.shape[1]              # 样本数量
    Y = Y.reshape(AL.shape)      # 保证Y与AL形状一致，便于计算

    # 计算输出层激活值的梯度，交叉熵损失对激活值AL的导数
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    
    # 计算最后一层（第L层）梯度，激活函数为sigmoid
    current_cache = caches[L - 1]   # 获取最后一层缓存，这是因为 Python 的列表索引是从 0 开始的，L-1才是最后一层，与数学计数不同
    grads["dA" + str(L)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, activation="sigmoid")
    
    # 反向循环计算前面L-1层的梯度，激活函数为ReLU
    for l in reversed(range(L - 1)):
        current_cache = caches[l]   # 当前层缓存
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(
            grads["dA" + str(l + 2)],   # 表示下一层激活梯度。l是 Python 索引，但dA的编号是数学层编号（1-based）
                                        # 当 l=2（Python索引）时，它实际上是数学上的第3层，而我们需要它的下一层的激活梯度，也就是第 4 层
                                        # 数学编号的转换公式就是：下一层的编号 =l（Python索引） + 2
            current_cache,
            activation="relu"
        )
        # 保存当前层的梯度
        grads["dA" + str(l + 1)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp

    return grads


In [60]:
# 获取测试数据，用于验证多层神经网络反向传播函数的正确性
AL, Y_assess, caches = L_model_backward_test_case()

# 计算反向传播的梯度
grads = L_model_backward(AL, Y_assess, caches)

# 打印第一层权重梯度dW1
print("dW1 = " + str(grads["dW1"]))

# 打印第一层偏置梯度db1
print("db1 = " + str(grads["db1"]))

# 打印第一层激活值梯度dA1
print("dA1 = " + str(grads["dA1"]))


dW1 = [[0.41010002 0.07807203 0.13798444 0.10502167]
 [0.         0.         0.         0.        ]
 [0.05283652 0.01005865 0.01777766 0.0135308 ]]
db1 = [[-0.22007063]
 [ 0.        ]
 [-0.02835349]]
dA1 = [[ 0.          0.52257901]
 [ 0.         -0.3269206 ]
 [ 0.         -0.32070404]
 [ 0.         -0.74079187]]


### 6.4 - 更新参数（Update Parameters）

在本节中，你将使用梯度下降法更新模型的参数：

$$ W^{[l]} = W^{[l]} - \alpha \text{ } dW^{[l]} \tag{16}$$  
$$ b^{[l]} = b^{[l]} - \alpha \text{ } db^{[l]} \tag{17}$$

其中，$\alpha$ 表示学习率（learning rate）。  
在计算出更新后的参数后，将它们存储到 `parameters` 字典中。


**练习**：实现 `update_parameters()` 函数，使用梯度下降法更新你的参数。

**提示说明**：  
对每一层的 $W^{[l]}$ 和 $b^{[l]}$ 使用梯度下降进行更新，其中 $l = 1, 2, ..., L$。


In [63]:
def update_parameters(parameters, grads, learning_rate):
    """
    使用梯度下降（Gradient Descent）更新神经网络的参数。
    
    参数：
    parameters -- Python 字典，包含当前神经网络的所有参数（权重 W 和偏置 b）
                  结构：{"W1": W1, "b1": b1, ..., "WL": WL, "bL": bL}
    grads -- Python 字典，包含由 L_model_backward() 反向传播得到的梯度
             结构：{"dW1": dW1, "db1": db1, ..., "dWL": dWL, "dbL": dbL}
    learning_rate -- 学习率（float），控制参数更新的步长
    
    返回：
    parameters -- Python 字典，更新后的参数
    """

    # len(parameters) 代表字典中键的数量，包含 W 和 b
    # 每一层有两个键：Wl 和 bl，因此总数除以 2 得到网络的层数 L
    L = len(parameters) // 2  # 神经网络的层数

    # 遍历每一层，按梯度下降公式更新参数
    for l in range(L):
        # 更新公式：Wl := Wl - α * dWl
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * grads["dW" + str(l + 1)]

        # 更新公式：bl := bl - α * dbl
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * grads["db" + str(l + 1)]
        
    # 返回更新后的参数字典
    return parameters


In [64]:
# 使用给定的测试用例生成初始参数和梯度
parameters, grads = update_parameters_test_case()

# 调用 update_parameters 函数，使用学习率 0.1 更新参数
parameters = update_parameters(parameters, grads, 0.1)

# 打印更新后的 W1
print("W1 = " + str(parameters["W1"]))

# 打印更新后的 b1
print("b1 = " + str(parameters["b1"]))

# 打印更新后的 W2
print("W2 = " + str(parameters["W2"]))

# 打印更新后的 b2
print("b2 = " + str(parameters["b2"]))


W1 = [[-0.59562069 -0.09991781 -2.14584584  1.82662008]
 [-1.76569676 -0.80627147  0.51115557 -1.18258802]
 [-1.0535704  -0.86128581  0.68284052  2.20374577]]
b1 = [[-0.04659241]
 [-1.28888275]
 [ 0.53405496]]
W2 = [[-0.55569196  0.0354055   1.32964895]]
b2 = [[-0.84610769]]


## 7 - 总结（Conclusion）

恭喜你成功实现了构建深层神经网络所需的全部函数！

我们知道这个练习任务比较长，但接下来会越来越轻松。接下来的部分会更简单一些。

在下一个练习中，你将把所有这些函数组合起来，构建两个模型：
- 一个两层的神经网络
- 一个 L 层的神经网络

你将实际使用这些模型来对猫和非猫的图像进行分类！
